In [ ]:
# https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/
# (1) from command line ----    git clone https://github.com/matterport/Mask_RCNN.git
#      after clone --- command line --- cd Mask_RCNN
#                              --- (sudo) python setup.py install

# (2) from command line --- get dataset --- git clone https://github.com/experiencor/kangaroo.git
# (3) download the model (below)
    # pip install wget
    #import wget
    #url = 'https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5'
    #wget.download(url, 'mask_rcnn_coco.h5')
    
# (4) pip install tensorflow==1.15.3 #important version 
# (5) # pip install keras==2.2.4    #important version
# (6) # # pip install mrcnn
# train a new object, for example, Kangaroo or something else 

import warnings
warnings.simplefilter("ignore")
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import os

In [ ]:
#Train/Test --- generate model 


class KangarooDataset(Dataset):
	def load_dataset(self, dataset_dir, is_train=True):
		self.add_class("dataset", 1, "clown")
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		for filename in listdir(images_dir):
			image_id = filename[:-4]
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

	def extract_boxes(self, filename):
		tree = ElementTree.parse(filename)
		root = tree.getroot()
		boxes = list()
		for box in root.findall('.//bndbox'):
			xmin = int(box.find('xmin').text)
			ymin = int(box.find('ymin').text)
			xmax = int(box.find('xmax').text)
			ymax = int(box.find('ymax').text)
			coors = [xmin, ymin, xmax, ymax]
			boxes.append(coors)
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	def load_mask(self, image_id):
		info = self.image_info[image_id]
		path = info['annotation']
		boxes, w, h = self.extract_boxes(path)
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			masks[row_s:row_e, col_s:col_e, i] = 1
			class_ids.append(self.class_names.index('clown'))
		return masks, asarray(class_ids, dtype='int32')

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

class KangarooConfig(Config):
	NAME = "kangaroo_cfg"
	NUM_CLASSES = 1 + 1
	STEPS_PER_EPOCH = 5

train_set = KangarooDataset()
train_set.load_dataset('/home/ubuntu/gitscripts/FobiaPhilter/training', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
test_set = KangarooDataset()
test_set.load_dataset('/home/ubuntu/gitscripts/FobiaPhilter/validation', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
config = KangarooConfig()
config.display()

model = MaskRCNN(mode='training', model_dir='./', config=config)
model.load_weights('/home/ubuntu/gitscripts/Mask_RCNN/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=1, layers='heads')

In [ ]:
# Evaluate the test dataset (predict vs. truth)

class KangarooDataset(Dataset):
	def load_dataset(self, dataset_dir, is_train=True):
		self.add_class("dataset", 1, "clown")
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		for filename in listdir(images_dir):
			image_id = filename[:-4]
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

	def extract_boxes(self, filename):
		tree = ElementTree.parse(filename)
		root = tree.getroot()
		boxes = list()
		for box in root.findall('.//bndbox'):
			xmin = int(box.find('xmin').text)
			ymin = int(box.find('ymin').text)
			xmax = int(box.find('xmax').text)
			ymax = int(box.find('ymax').text)
			coors = [xmin, ymin, xmax, ymax]
			boxes.append(coors)
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	def load_mask(self, image_id):
		info = self.image_info[image_id]
		path = info['annotation']
		boxes, w, h = self.extract_boxes(path)
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			masks[row_s:row_e, col_s:col_e, i] = 1
			class_ids.append(self.class_names.index('clown'))
		return masks, asarray(class_ids, dtype='int32')

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

class PredictionConfig(Config):
	NAME = "kangaroo_cfg"
	NUM_CLASSES = 1 + 1
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1


def evaluate_model(dataset, model, cfg):
    APs = list()
    for image_id in dataset.image_ids:
        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
         scaled_image = mold_image(image, cfg)
        sample = expand_dims(scaled_image, 0)
        yhat = model.detect(sample, verbose=0)
        r = yhat[0]
        AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
        APs.append(AP)
    mAP = mean(APs)
    return mAP

test_set = KangarooDataset()
test_set.load_dataset('/home/ubuntu/gitscripts/FobiaPhilter/validation', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
cfg = PredictionConfig()
model = MaskRCNN(mode = 'inference', model_dir = './', config = cfg)
model.load_weights('/home/ubuntu/gitscripts/FobiaPhilter/kangaroo_cfg20200922T1734/mask_rcnn_kangaroo_cfg_0001.h5', by_name = True)
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)

In [ ]:
# validate in new images (no annotation), save the images with BB 

def main(): 
    for count, filename in enumerate(os.listdir("/home/ubuntu/gitscripts/FobiaPhilter/Frames/images/")):
        newfilename = ''.join([n for n in filename if n.isdigit()]) + ".jpg"
        src = '/home/ubuntu/gitscripts/FobiaPhilter/Frames/images/' + filename
        dst ='/home/ubuntu/gitscripts/FobiaPhilter/Frames/images/' + newfilename    
        os.rename(src, dst) 
         
if __name__ == '__main__': 
    main() 

    
class KangarooDataset(Dataset):
	def load_dataset(self, dataset_dir, is_train=True):
		self.add_class("dataset", 1, "clown")
		images_dir = dataset_dir + '/images/'    
		l = os.listdir('/home/ubuntu/gitscripts/FobiaPhilter/Frames/images/')
		lsorted = sorted(l,key=lambda x: int(os.path.splitext(x)[0]))
		for filename in lsorted:
			image_id = filename[:-4]
			img_path = images_dir + filename
			self.add_image('dataset', image_id=image_id, path=img_path)

	def extract_boxes(self, filename):
		root = ElementTree.parse(filename)
		boxes = list()
		for box in root.findall('.//bndbox'):
			xmin = int(box.find('xmin').text)
			ymin = int(box.find('ymin').text)
			xmax = int(box.find('xmax').text)
			ymax = int(box.find('ymax').text)
			coors = [xmin, ymin, xmax, ymax]
			boxes.append(coors)
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	def load_mask(self, image_id):
		info = self.image_info[image_id]
		path = info['annotation']
		boxes, w, h = self.extract_boxes(path)
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			masks[row_s:row_e, col_s:col_e, i] = 1
			class_ids.append(self.class_names.index('clown'))
		return masks, asarray(class_ids, dtype='int32')

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

class PredictionConfig(Config):
	NAME = "kangaroo_cfg"
	NUM_CLASSES = 1 + 1
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1
 

# plot a number of photos with ground truth and predictions
def plot_predicted_new(dataset, model, cfg, n_images=765):
	for i in range(n_images):
		image = dataset.load_image(i)
		scaled_image = mold_image(image, cfg)
		sample = expand_dims(scaled_image, 0)
		yhat = model.detect(sample, verbose=0)[0]
		fig = pyplot.figure()
		pyplot.imshow(image)
		pyplot.title('Predicted')
		ax = pyplot.gca()
		for box in yhat['rois']:
			y1, x1, y2, x2 = box
			width, height = x2 - x1, y2 - y1
			rect = Rectangle((x1, y1), width, height, fill=False, color='red')
			ax.add_patch(rect)     
		pyplot.show()
		fig.savefig(f'/home/ubuntu/gitscripts/FobiaPhilter/images_pred_box/{i}.jpg')

test_set = KangarooDataset()
test_set.load_dataset('/home/ubuntu/gitscripts/FobiaPhilter/Frames', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
cfg = PredictionConfig()
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
model_path = '/home/ubuntu/gitscripts/FobiaPhilter/kangaroo_cfg20200922T1734/mask_rcnn_kangaroo_cfg_0001.h5'
model.load_weights(model_path, by_name=True)
plot_predicted_new(test_set, model, cfg)